In [ ]:
#Library
import pandas as pd
import numpy as np
import datetime as dt
import string
import time
import matplotlib.pyplot as plt
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.piecewise import PiecewiseAggregateApproximation
from tslearn.piecewise import SymbolicAggregateApproximation, OneD_SymbolicAggregateApproximation
from math import sqrt
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from langdetect import detect
import matplotlib.pyplot as plt
import seaborn as sns
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


In [ ]:

# Computing covariance and correlation matrix
df = pd.read_csv('price_df.csv',index_col = 0)
df['returns'] = df.groupby(['name'])['close'].diff()
print(df)
len(df['name'].unique().tolist())
df_transposed = df.set_index(['name','day']).close.unstack('name')
df_return = df_transposed.pct_change()
df_return.cov()
df_return.corr()

groups = df.groupby(['name'])
groups.get_group('AIRPORT-C8')
groups.get_group('3Month') # have full records for 3 months
df_new = groups.filter(lambda x : len(x)==60) #select 60 stock
df_transposed1 = df_new.set_index(['name','day']).close.unstack('name')
df_return1 = df_transposed1.pct_change()
df_return1.cov()
df_return1.corr()

#positive and negative correlation
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_pos_correlations(df, n):
    au_corr = df.corr().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

def get_top_neg_correlations(df, n):
    au_corr = df.corr().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=True)
    return au_corr[0:n]

#Positive correlated stocks
x = get_top_pos_correlations(df_return, n)
y = get_top_pos_correlations(df_return1, n)

print("Top %d Positive Correlations" % n)
print("For different range of records: ");print(x)
print("Top %d Positive Correlations" % n)
print(" For stocks with >60 records: " );print(y)

#Negative correlated stocks
r = get_top_neg_correlations(df_return, n)
s = get_top_neg_correlations(df_return1, n)
    
print("Top %d Negative Correlations" % n)
print("For different range of records: ");print(r)
print("Top %d Negative Correlations" % n)
print(" For stocks with >60 records: " );print(s)


#Find stock returns and Risk
dfReturns = df_transposed1.apply(lambda x: np.log(x) - np.log(x.shift(1))).mean()*61
dfReturns = pd.DataFrame(dfReturns)
dfReturns.columns = ['Returns']
dfReturns['Risk'] = df_transposed1.pct_change().std()*sqrt(61)

#find correlation matrix between each stock
corr = df_transposed1.corr()

# generate the linkage matrix for clustering
Z = linkage(corr, 'average')

# function to create dendogram
def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (truncated)')
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata

# Hclustering diagram
max_d = 7 
fancy_dendrogram(
    Z,
    truncate_mode='lastp',
    p=12,
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,
    annotate_above=10, 
    max_d=max_d,
)
plt.show()

max_d = max_d
clusters = fcluster(Z, max_d, criterion='distance')
cluster_list = clusters.tolist()
dfReturns['clusters'] = cluster_list
from collections import Counter
Counter(clusters)

# plot clusters results with regression line
%matplotlib inline
facet = sns.lmplot(data=dfReturns, x='Risk', y='Returns', hue='clusters',palette='Set2',fit_reg=True, legend=True, legend_out=True)

clusterext = dfReturns[dfReturns['clusters']==5]
print(clusterext)
stocknames = clusterext.index.tolist()

clusterhighext = dfReturns[dfReturns['clusters']==3]
print(clusterhighext)
highstocknames = clusterhighext.index.tolist()

clusterlowext = dfReturns[dfReturns['clusters']==1]
print(clusterlowext)
lowstocknames = clusterlowext.index.tolist()

df_new2 = pd.DataFrame()
df_new2 = df_new[df_new['name'].isin(stocknames)]
df_new2 = df_new2.dropna()
df_new2.to_csv('stocksmidrisk.csv',index=False)

df_new3 = pd.DataFrame()
df_new3 = df_new[df_new['name'].isin(highstocknames)]
df_new3 = df_new3.dropna()
df_new3.to_csv('stockshighrisk.csv',index=False)

df_new4 = pd.DataFrame()
df_new4 = df_new[df_new['name'].isin(lowstocknames)]
df_new4 = df_new4.dropna()
df_new4.to_csv('stockslowrisk.csv',index=False)


# =============================================================================
# Part 3: Identify valuable stocks - potential stocks for investment
# =============================================================================

# Price to Earnings Ratio
df_fin = pd.read_csv('financial_df.csv')
df_fin.rename(columns={'stockname':'name'}, inplace=True)
df_fin = df_fin[['name','quarter','price','pbt','eps','YoY']]   

df_low = pd.read_csv('stockslowRisk.csv')
df_low['risk'] = 'low risk'
low_fin = set(df_low['name']).intersection(set(df_fin['name']))
len(low_fin)
low_fin_stocks = list(low_fin)
df_low_fin = pd.merge(df_low,df_fin,how='inner',on='name')
df_low_fin.to_csv('stockslowfin.csv',index=False)

df_mid = pd.read_csv('stocksmidrisk.csv')
df_mid['risk'] = 'mid risk'
mid_fin = set(df_mid['name']).intersection(set(df_fin['name']))
len(mid_fin) 
mid_fin_stocks = list(mid_fin)
df_mid_fin = pd.merge(df_mid,df_fin,how='inner',on='name')
df_mid_fin.to_csv('stocksmidfin.csv',index=False)

df_hi = pd.read_csv('stockshighrisk.csv')
df_hi['risk'] = 'high risk'
hi_fin = set(df_hi['name']).intersection(set(df_fin['name']))
len(hi_fin)
hi_fin_stocks = list(hi_fin)
df_hi_fin = pd.merge(df_hi,df_fin,how='inner',on='name')
df_hi_fin.to_csv('stockshifin.csv',index=False)

Alldf = pd.concat([df_low_fin,df_mid_fin,df_hi_fin], ignore_index=True)
len(Alldf['name'].unique()) #143 companies in total
Alldf.to_csv('combinedstockriskfinancial.csv',index=False)

Alldf = pd.read_csv('combinedstockriskfinancial.csv')
df_fin = Alldf[['name','risk','quarter','price','pbt','eps','YoY']]
df_fin.drop_duplicates(subset=['name','price'],keep='first', inplace=True)

# potential stocks filter
investstock = df_fin[(df_fin['pbt_norm'] >= 0) & 
                     (df_fin['PE_ratio'] >= 0) &
                     (df_fin['PE_ratio'] <=3) &
                     (df_fin['YoY_norm'] >-1)]

invstocknames = investstock['name'].tolist()
len(invstocknames)
print(invstocknames)

Alldf['strategy'] = Alldf['name'].apply(lambda x: 'potentialgain' if x in invstocknames else 'potentialloss')
newdf = Alldf[Alldf['strategy']=='potentialgain']

newdf.drop(['open','quarter','price','pbt','eps','YoY','high','low','strategy'], axis=1, inplace=True)
len(newdf['name'].unique().tolist())
newdf.to_csv('potentialstock.csv',index=False)
